# Duplicate two-sided reactions in a SBML model

A requirement for neural computations with metabolic networks using AMNs is the positivity of all fluxes.

This notebook shows the steps to transforms a SBML model into an *AMN-compatible* SBML model where all exchange reactions are possible in both ways, and reversible internal reactions are duplicated for each way. Added to that, the reactions encoded as backward are recoded in the other way, the forward way. In this transformed model, we also add a suffix "i", for inflowing, and "o" for outflowing reactions. It is further described what we consider "inflow" and "outflow".

# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [2]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn.yml
    !mamba env update -n base -f environment_amn.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Duplicate_Model import *

['README.md', 'Duplicate_Model.ipynb', 'Build_Model_Dense.ipynb', 'Build_Dataset.py', 'Dataset_experimental', '.ipynb_checkpoints', '.git', 'Build_Experimental.ipynb', 'Reservoir', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'Duplicate_Model.py', 'LICENSE', 'Build_Dataset.ipynb', 'Dataset_input', '__pycache__', 'Build_Experimental.py', 'old', 'environment_amn.yml', 'Build_Model.py', 'Build_Model.ipynb', '.DS_Store']


## Download and inspect the model

Run this cell for E. coli core model

In [18]:
model_path = "Dataset_input/e_coli_core.xml"
url = "http://bigg.ucsd.edu/static/models/e_coli_core.xml"
response = requests.get(url)
open(model_path, "wb").write(response.content)
model = cobra.io.read_sbml_model(model_path)

Run this cell for iML1515 model

In [19]:
model_path = "Dataset_input/iML1515.xml"
url = "http://bigg.ucsd.edu/static/models/iML1515.xml"
response = requests.get(url)
open(model_path, "wb").write(response.content)
model = cobra.io.read_sbml_model(model_path)

First, we can get some basic informations on the model.

model.boundary gets all reactions that introduce or remove matter in the system (inflows, outflows)

model.medium lists all reactions of model.boundary that are reversible (irreversible are outflows/sinks)

model.objective is the expression to be optimized by default

In [20]:
print(model.boundary)
print(model.medium)
print(model.objective)

[<Reaction EX_pi_e at 0x7fd0d3c3ae10>, <Reaction EX_co2_e at 0x7fd0d3c3ed10>, <Reaction EX_met__L_e at 0x7fd0d3c3e390>, <Reaction EX_metsox_S__L_e at 0x7fd0d3bc57d0>, <Reaction EX_acgam_e at 0x7fd0d3bd1e10>, <Reaction EX_cellb_e at 0x7fd0d3bda250>, <Reaction EX_crn_e at 0x7fd0d3bda2d0>, <Reaction EX_hxan_e at 0x7fd0d3bdb710>, <Reaction EX_ile__L_e at 0x7fd0d3bdaa10>, <Reaction EX_chol_e at 0x7fd0d3bdc4d0>, <Reaction EX_fe3_e at 0x7fd0d3bdc7d0>, <Reaction EX_lac__L_e at 0x7fd0d3bdc690>, <Reaction EX_leu__L_e at 0x7fd0d3bddc90>, <Reaction EX_glcn_e at 0x7fd0d3bde250>, <Reaction EX_no3_e at 0x7fd0d3bde850>, <Reaction EX_h_e at 0x7fd0d3be0550>, <Reaction EX_orn_e at 0x7fd0d3be0a10>, <Reaction EX_gln__L_e at 0x7fd0d3be2850>, <Reaction EX_pro__L_e at 0x7fd0d3be3a90>, <Reaction EX_glyc_e at 0x7fd0d3be4d90>, <Reaction EX_man_e at 0x7fd0d3be55d0>, <Reaction EX_ade_e at 0x7fd0d3be6590>, <Reaction EX_mn2_e at 0x7fd0d3be1e10>, <Reaction EX_4abut_e at 0x7fd0d3be8990>, <Reaction EX_ac_e at 0x7fd0d3b

# Checking the objective of the model
Optional step for exploring how the biomass is encoded. In some models, several biomass reactions are available and one has to make sure using the right one.

In [21]:
for reac in model.reactions:
     if "biomass" in reac.id or "BIOMASS" in reac.id:
        print(reac)

BIOMASS_Ec_iML1515_core_75p37M: 0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.000223 2ohph_c + 0.00026 4fe4s_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 2.5e-05 cobalt2_c + 0.133508 ctp_c + 0.000709 cu2_c + 0.09158 cys__L_c + 0.026166 datp_c + 0.027017 dctp_c + 0.027017 dgtp_c + 0.026166 dttp_c + 0.000223 fad_c + 0.006715 fe2_c + 0.007808 fe3_c + 0.26316 gln__L_c + 0.26316 glu__L_c + 0.612638 gly_c + 0.215096 gtp_c + 70.028756 h2o_c + 0.094738 his__L_c + 0.290529 ile__L_c + 0.195193 k_c + 0.019456 kdo2lipid4_e + 0.450531 leu__L_c + 0.343161 lys__L_c + 0.153686 met__L_c + 0.008675 mg2_c + 0.000223 mlthf_c + 0.000691 mn2_c + 7e-06 mobd_c + 0.013894 murein5px4p_p + 0.001831 nad_c + 0.000447 nadp_c + 0.013013 nh4_c + 0.000323 ni2_c + 0.063814 pe160_p + 0.075214 pe161_p + 0.185265 phe__L_c + 0.000223 pheme_c + 0.221055 pro__L_c + 0.000223 pydx5p_c + 0.000223 r

# Screen outflowing and inflowing reactions

For each reaction that has different compartments in reactants and products (we call it "transfer reactions"), we annotate the reaction with a suffix "i" for inflowing (None --> e --> p --> c --> m) and "o" for outflowing (m --> c --> p --> e --> None). When the compartment-changing of metabolites is balanced, or not present, we use different suffix: "for" as in forward, designating the default way of the reaction (positive flux) and "rev" as in reverse, designating the opposite way (negative flux). We reverse the products and reactants so that the same reactions happen, ensuring that we have a positive flux for all reactions.

To do so, we first define a dictionary for mapping which (reactant compartment, product compartment) pair is matching which suffix: "io_dict".

Some reactions are problematic because they are showing both inflow and outflow simultaneously. To tackle this, we ignore the small molecules listed in "unsignificant_mols".

For each reaction, we count the number of "inflowing" and "outflowing" pairs, and the way the reaction happens (forward, backward, reversible or other).

In [22]:
io_dict = {"_i": [(None, "e"), (None, "c"), ("e","p"), ("p", "c"), ("e", "c"), ("c", "m"), ("p", "m")],
           "_o": [("c", None), ("e", None), ("p", "e"), ("c", "p"), ("c", "e"), ("m", "c"), ("m", "p")]}

unsignificant_mols = ["h_p", "h_c", "pi_c", "pi_p", "adp_c", "h2o_c", "atp_c"]

# Will print a dictionary counting the reactions in reversible, forward, backward
reac_id_to_io_count_and_way = screen_out_in(model, io_dict, unsignificant_mols)

# To uncomment in order to see the structure of the screening dictionary
# print(reac_id_to_io_count_and_way)

{'r': 663, 'f': 2047, 'b': 0, 'o': 2}


# Duplicate reactions

Here we make a copy of the model,  named 'new_model', then perform the duplication of appropriate reactions.

We duplicate all exchange reactions (excepted sink reactions) and reversible internal reactions (not unidirectional ones). We get the suffix '_i' for compartment changing reaction from the exterior to the cytoplasm, and '_o' for the other way. We also use the suffix "_for" and "_rev" for forward and reverse duplicated reactions that do not change compartment, or show equal compartment exchanges.

In [23]:
new_model = duplicate_model(model, reac_id_to_io_count_and_way)

The default model had 2712 reactions and the duplicated-reactions model has 3682 reactions.


# Lower bounds check-up

Here we simply check which reactions have a non-zero lower bound

In [24]:
for reac in new_model.reactions:
    if reac.lower_bound != 0:
        print('reaction with non-zero lower bound:', reac.id, reac.bounds)
for el in new_model.medium:
    if new_model.reactions.get_by_id(el).lower_bound != 0:
        print('medium reaction with non-zero lower bound:',el)

reaction with non-zero lower bound: ATPM (6.86, 1000.0)


# Get the default medium back on the duplicated-reactions model

The new model with duplicated exchange reactions is badly handled by COBRA for the medium object generation: all inflowing reactions are inside the medium object.

There is a problem to be corrected: all upper bounds are set at 1000 by default. If we change this upper bound to 0 when we duplicate reactions, we get an empty medium object.

The solution is to correct the medium of the duplicated model, all non-default medium exchange reactions put at 1e-300, so they are at a value very close to 0 but still appear in the medium object.

In [25]:
default_med = model.medium
new_med = new_model.medium
correct_med =  correct_duplicated_med(default_med, new_med)
new_model.medium = correct_med
print(new_model.medium)

1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
10.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
{'EX_pi_e_i': 1000.0, 'EX_co2_e_i': 1000.0, 'EX_met__L_e_i': 1e-300, 'EX_metsox_S__L_e_i': 1e-300, 'EX_acgam_e_i': 1e-300, 'EX_cellb_e_i': 1e-300, 'EX_crn_e_i': 1e-300, 'EX_hxan_e_i': 1e-300, 'EX_ile__L_e_i': 1e-300, 'EX_chol_e_i': 1e-300, 'EX_fe3_e_i': 1000.0, 'EX_lac__L_e_i': 1e-300, 'EX_leu__L_e_i': 1e-300, 'EX_glcn_e_i': 1e-300, 'EX_no3_e_i': 1e-300, 'EX_h_e_i': 1000.0, 'EX_orn_e_i': 1e-300, 'EX_gln__L_e_i': 1e-300, 'EX_pro__L_e_i': 1e-300, 'EX_glyc_e_i': 1e-300, 'EX_man_e_i': 1e-300, 'EX_ade_e_i': 1e-300, 'EX_mn2_e_i': 1000.0, 'EX_4abut_e_i': 1e-300, 'EX_ac_e_i': 1e-300, 'EX_akg_e_i': 1e-300, 'EX_ala__L_e_i': 1e-300, 'EX_arg__L_e_i': 1e-300, 'EX_asp__L_e_i': 1e-300, 'EX_pyr_e_i': 1e-300, 'EX_succ_e_i': 1e-300, 'EX_thymd_e_i': 1e-300, 'EX_rib__D_e_i': 1e-300, 'EX_tyr__L_e_i': 1e-300, 'EX_cytd_e_i': 1e-300, 'EX_dcyt_e_i

# Medium check-up (default model V.S. duplicated-reaction model)

Here we compare the results with randomized medium objects for both models, reporting the absolute difference between the two.

In [26]:
for i in range(10):
    s, new_s = change_medium(model, new_model, i*3)
    if s != None and new_s != None:
        print(s, new_s, "diff = ", abs(s-new_s))
    elif s != None:
        print("infeasible duplicated medium")
    elif new_s != None:
        print("infeasible default medium")
    elif s == None and new_s == None:
        print("Both medium are impossible")

0.3692496473883969 0.36924964738840355 diff =  6.661338147750939e-15
0.08745529196942173 0.08745529196942319 diff =  1.457167719820518e-15
0.45251524105939067 0.45251524105939894 diff =  8.271161533457416e-15
0.3520250542783163 0.35202505427832625 diff =  9.936496070395151e-15
0.7873623549879809 0.7873623549880019 diff =  2.098321516541546e-14
0.25037920551964943 0.2503792055196539 diff =  4.496403249731884e-15
0.5755530272256144 0.5755530272256257 diff =  1.1324274851176597e-14
0.24708666582567942 0.24708666582568484 diff =  5.412337245047638e-15
0.8737974348363089 0.8737974348363232 diff =  1.432187701766452e-14
0.7805728235038698 0.7805728235038877 diff =  1.787459069646502e-14


# Saving the duplicated-reactions model

In [27]:
new_model.repair() # rebuild indices and pointers in the model if necessary

In [28]:
print("Original model's location: " + model_path)
new_name = model_path[:-4] + "_duplicated" + model_path[-4:]
cobra.io.write_sbml_model(new_model, new_name)
print("Duplicated model's location: " + new_name)

Original model's location: Dataset_input/iML1515.xml
Duplicated model's location: Dataset_input/iML1515_duplicated.xml
